In [4]:
import pandas as pd
import lyricsgenius as lg
import re
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax
import numpy as np
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import csv
import urllib.request
from gensim.parsing.preprocessing import remove_stopwords




/Users/adamo/Documents/data_project/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

client_access_token = ''

LyricsGenius = lg.Genius(client_access_token)
song = LyricsGenius.search_song("Eminem", "Stan")

Searching for "Eminem" by Stan...
Done.


In [3]:
lyrics = song.lyrics
#lyrics = lyrics.split("lyrics", 1)[-1]
lyrics

'647 ContributorsTranslationsTürkçeDeutschMagyarPortuguêsFrançaisItalianoSvenskaРусскийStan Lyrics[Produced by DJ Mark the 45 King]\n[Intro: Dido]\nMy tea\'s gone cold, I\'m wondering why I\nGot out of bed at all\nThe morning rain clouds up my window\nAnd I can\'t see at all\nAnd even if I could, it\'d all be grey\nBut your picture on my wall\nIt reminds me that it\'s not so bad, it\'s not so bad\n[Chorus: Dido]\nMy tea\'s gone cold, I\'m wondering why I\nGot out of bed at all\nThe morning rain clouds up my window\nAnd I can\'t see at all\nAnd even if I could, it\'d all be grey\nBut your picture on my wall\nIt reminds me that it\'s not so bad, it\'s not so bad\n[Verse 1: Eminem]\nDear Slim, I wrote you, but you still ain\'t callin\'\nI left my cell, my pager and my home phone at the bottom\nI sent two letters back in autumn, you must not\'ve got \'em\nThere probably was a problem at the post office or somethin\'\nSometimes I scribble addresses too sloppy when I jot \'em\nBut anyways, f

In [4]:
def clean_text(text):

    text = text.lower()
    text = text.split("lyrics", 1)[-1]
    text = text.replace('\n', ' ')
    text = text.replace('\\', '')
    text = re.sub(r'[\'\",\.!?\\]', '', text)
    text = re.sub(r'\[.*?\]', ' ', text) 
    text = re.sub(r'\w*\d\w*', ' ', text)
    text = re.sub(r'[()]', ' ', text)
    
    return text

In [5]:
lyrics = clean_text(lyrics)
lyrics

'    my teas gone cold im wondering why i got out of bed at all the morning rain clouds up my window and i cant see at all and even if i could itd all be grey but your picture on my wall it reminds me that its not so bad its not so bad   my teas gone cold im wondering why i got out of bed at all the morning rain clouds up my window and i cant see at all and even if i could itd all be grey but your picture on my wall it reminds me that its not so bad its not so bad   dear slim i wrote you but you still aint callin i left my cell my pager and my home phone at the bottom i sent two letters back in autumn you must notve got em there probably was a problem at the post office or somethin sometimes i scribble addresses too sloppy when i jot em but anyways fuck it whats been up man hows your daughter my girlfriends pregnant too im bout to be a father if i have a daughter guess what ima call her ima name her bonnie i read about your uncle ronnie too im sorry i had a friend kill himself over som

In [1]:
# Removing stop words

# Remove stopwords using Gensim
lyrics = remove_stopwords(lyrics)
 


NameError: name 'remove_stopwords' is not defined

In [7]:
# Setting up model

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]


model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [31]:
lyrics = """When I die, fuck it, I wanna go to hell
'Cause I'm a piece of shit, it ain't hard to fuckin' tell (What you talkin' 'bout, man?)
It don't make sense, goin' to heaven with the goodie-goodies
Dressed in white, I like black Timbs and black hoodies (Aw, man)"""

In [32]:
encoded_input = tokenizer(lyrics, return_tensors='pt', max_length=512)
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) negative 0.8035
2) neutral 0.1621
3) positive 0.0344


In [19]:
from transformers import pipeline


sentiment_task = pipeline("sentiment-analysis", model=f"cardiffnlp/twitter-roberta-base-sentiment-latest", tokenizer=f"cardiffnlp/twitter-roberta-base-sentiment-latest")
sentiment_task(lyrics)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: The expanded size of the tensor (645) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 645].  Tensor sizes: [1, 514]